# Problema 1
$$
\begin{array}{ll}
\operatorname{minimizar} & \frac{1}{2} \vec{\boldsymbol{x}}^{\top} G \vec{\boldsymbol{x}}+\vec{\boldsymbol{c}}^{\top} \vec{\boldsymbol{x}} \\
\text { sujeto a } & A \vec{x}=\vec{\boldsymbol{b}} \\
& x_{i} \geq \ell_{i} \quad \text { si } \ell_{i} \text { es finito, } \\
& x_{i} \leq u_{i} \quad \text { si } u_{i} \text { es finito. }
\end{array}
$$

- Definimos $J \subset I$ como sigue:

- Para $x_{j} \geq \ell_{j}$ definimos $\left|g_{j}\left(x_{0}\right)\right| \leq 8 \varepsilon_{m} \max \left\{\left|\ell_{j}\right|, 1\right\} \Longrightarrow j \in J$

- Para $x_{j} \leq u_{j}$ definimos $\left|g_{j}\left(x_{0}\right)\right| \leq 8 \varepsilon_{m} \max \left\{\left|u_{j}\right|, 1\right\} \Longrightarrow j \in J$

Donde $\varepsilon_{m}$ es el épsilon de la máquina `eps(Float64)`

In [1]:
include("Solvers.jl")
using .Solvers, LinearAlgebra, MAT, .Solvers.Utils

In [2]:
problem = matread("../data/lpi_woodinfe.mat")["Problem"]

Dict{String, Any} with 11 entries:
  "A"      => …
  "aux"    => Dict{String, Any}("c"=>[10.0; 11.0; … ; 15.0; 15.0], "hi"=>[100.0…
  "b"      => [0.0; 0.0; … ; 0.0; 0.0]
  "date"   => "1989"
  "name"   => "LPnetlib/lpi_woodinfe"
  "kind"   => "linear programming problem"
  "author" => "H. Greenberg"
  "id"     => 731.0
  "notes"  => ["An infeasible Netlib LP problem, in lp/infeas.  For more inform…
  "title"  => "Netlib LP problem woodinfe: minimize c'*x, where Ax=b, lo<=x<=hi"
  "ed"     => "J. Chinneck"

In [3]:
b = problem["b"] 
c = problem["aux"]["c"][:] # El [:] es para interpretar como vector
l = problem["aux"]["lo"][:] # El [:] es para interpretar como vector
u = problem["aux"]["hi"][:] # El [:] es para interpretar como vector
n_eq = length(b)
A_eq = problem["A"]
G = I(length(c))

# Como all(isfinite.(l)) == true, se complen todas las cotas inferiores
A = [A_eq; -I(length(l))]
b = [b; l]

# Imponiendo cotas superiores
# Concatenando las filas I_j de una identidad tal que u_j < Inf
mask = isfinite.(u)
A = [A; I(length(u))[mask, :]]
# Igual cambiando b para que dimensiones coincidan
b = [b; u[mask]]
b = b[:]

138-element Vector{Float64}:
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   0.0
   ⋮
 200.0
 100.0
  25.0
  10.0
  50.0
  40.0
  20.0
   5.0
  30.0

In [4]:
# Definiendo el conjunto inicial de igualdades
# Obtenemos x_0 con linprog (pero ya sabemos que va a ser vec(0))
x_0 = linprog(A, b, n_eq)

# Shorthand para epsilon de maquina
εₘ = eps(Float64)

# Obteniendo los masks que indican si se usa la restricción
# Recordamos que para las cotas inferiores g_i(x) = l_i - x_i & para las sup g_i(x) = x_i - u_i
J_l = abs.(l - x_0) .<= 8 * εₘ *  max.(abs.(l), ones(length(l)))

# Ahora para las superiores, recordando quitar los renglones donde u_j no es finito (por size de A).
J_u = abs.(x_0 - u) .<= 8 * εₘ *  max.(abs.(u), ones(length(u)))
J_u = J_u[isfinite.(u)]

# Concatenando
W_0 = [trues(n_eq); J_l; J_u]

138-element BitVector:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 0
 0
 0
 0
 0
 0
 0
 0
 0

Nótese que el número de filas de W_0 coincide con el de A, y el tamaño de b

In [5]:
@assert size(A, 1) == length(W_0)
@assert size(A, 1) == length(b)

In [6]:
x_star, _, μ = activeSetMethod(G, c, A, b, n_eq, W_0)

Rama 2. j = 19, μ=0.0 


([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [3.857142857142855, -1.0065053380614282e-14, 2.9999999999999933, -1.2960903596640766e-14, 1.333333333333324, -4.811065295335972e-15, 3.9999999999999902, 2.107206423394375e-15, 8.666666666666664, -1.7073028880898917e-14  …  2.7270194271633698e-15, 2.1666666666666665, -1.801574965604803e-14, 5.195270059977747e-16, 4.1005617914360615e-15, -2.5999999999999885, 4.799596855932396e-15, 5.4527954440951835e-15, 1.3333333333333388, -0.0], [13.857142857142863, 13.999999999999986, 13.333333333333337, 19.000000000000007, 19.66666666666666, 15.85714285714285, 18.000000000000014, 13.333333333333336, 48.80000000000002, 46.39999999999997  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [12]:
x_star' * G * x_star + c' * x_star

0.0